In [2]:
!pip install rasterio

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
!pip3 install hub==1.1.1

     |████████████████████████████████| 100 kB 705 kB/s ta 0:00:01
     |████████████████████████████████| 328 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 125 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 1.8 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 1.4 MB/s  eta 0:00:01
     |████████████████████████████████| 296 kB 10.7 MB/s eta 0:00:01


     |████████████████████████████████| 69 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 10.4 MB/s eta 0:00:01
  Created wheel for outdated: filename=outdated-0.2.0-py3-none-any.whl size=4960 sha256=575de1f606897724fd6a0c59dfdf6e66da8fd9290b385b66effff1328dc30db5
  Stored in directory: /root/.cache/pip/wheels/6f/cd/a2/e49170b2cf59e88b952f3414f25a54d9f16f033bded4aaab26
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=10a1553ef7f91b296c658df7c298ad7e01982d04c08e95585947575e712faeab
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5035 sha256=b1e089f725497792e9dd1e0e68cd81377d22d254ed73ec4d0c4697cf6faaa834
  Stored in directory: /root/.cache/pip/wheels/12/1c/38/0def51e15add93bff3f4bf9c248b94db0839b980b8535e72a0
Succes

In [4]:
# Login to hub
!hub login -u 'rasha' -p 'AIEngineer4life'

In [5]:
import hub
from hub import Dataset
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from matplotlib import pyplot
from hub.schema import Image
from hub.schema import Tensor
from hub.schema import Mask
import os
from PIL import Image as Img
import cv2

In [32]:
# Create the dataset schem on hub (Only once)
tag = "rasha/SpaceNet_2_Las_Vegas"
ds = {}
ds = Dataset(
    tag,
    shape=(3850,),
   mode = "w+",  
  schema = {
      
       "rgbs": Image(shape=(None, None, 3), dtype="uint8", max_shape=(900, 900, 3)),
        "masks": Image(shape=(None, None), dtype="uint8", max_shape=(900, 900))
            
    },
)

In [33]:
img_dir = '../input/spacenet-2-lasvegas/SP_2_LV/Vegas_RGB/'
mask_dir = '../input/spacenet-2-lasvegas/SP_2_LV/binary_masks/'


# Get dictionary for data and corresponding mask
data = []
def get_data(root):
    """Create list of dictionary for each image and mask"""
    for file in os.listdir(os.path.join(root)):
        if os.path.isfile(os.path.join(root, file)):
            img = cv2.imread(os.path.join(root, file)).astype('uint8')
            mask = get_mask(file)               
            img_target = {}
            img_target = {
                'rgb' : img,
                'mask': mask
            }
            # Dictionary of image and its target in numpy format
            data.append(img_target)
    return data  

def get_mask(file, mask_root=mask_dir):
    """Get the corresponding mask for an image"""
    mask_name = file[0:file.index('PS')] + 'mask_buildings' + file[file.index('_img'):] 
    mask = cv2.imread(os.path.join(mask_dir, mask_name), 0).astype('uint8')
    #mask_ = np.expand_dims(mask,2)
    return mask
    

In [34]:
vegas_dataset = get_data(img_dir)

In [35]:
imgs = []
masks = []

for pairs in vegas_dataset:
    imgs.append(pairs['rgb'])
    masks.append(pairs['mask'])

In [36]:
print(masks[10].dtype)
print(masks[10].shape)

uint8
(650, 650)


In [37]:
for i in range(len(masks)):
    ds["rgbs"][i] = imgs[i]
    ds["masks"][i] = masks[i]



In [39]:
# Upload to hub
ds.commit()

In [ ]:
# Load data
ds = Dataset('rasha/SpaceNet_2_Las_Vegas')

# Use .numpy() to get the numpy array of the element
# Bands start from 0 here 
input = (ds["rgbs"][1].numpy()).astype('uint8')
mask = (ds["masks"][1].numpy()).astype('uint8')
pyplot.imshow(input)
